In [1]:
#to get auto matic reloading and inline editing 
%reload_ext autoreload
%autoreload 2 
%matplotlib inline

#my fast ai libraries
from fastai.imports import  * 
from fastai.transforms import *
from fastai.torch_imports import *
from fastai.conv_learner import * 
from fastai.model import *
from fastai.dataset import * 
from fastai.sgdr import *
from fastai.plots import * 
torch.cuda.set_device(0)

In [2]:
PATH = "data/dogbreeds/"
sz = 224
bs = 58
arch = resnext101_64

In [3]:
label_csv = f'{PATH}labels.csv'# load csv and get validation set
n = len(list(open(label_csv))) 
val_idx = get_cv_idxs(n)  

In [ ]:
!ls {PATH}

In [ ]:
label_df = pd.read_csv(label_csv) # get header of the cvs file, see whats in it 
label_df.head()

In [ ]:
"""
createa a pivot table to see what data is in there, sorting by length 
breed is the main index 
"""
label_df.pivot_table(index='breed', aggfunc=len).sort_values('id', ascending=False) 

In [5]:
"""Here is the function to get the data from teh CSV file
We can first give it the path for the CSV, the folder to use, 
test_name (for kaggle), validation indexes, suffix if 
there is none in the data transforms and batch size
"""
def getdata(sz, bs):
    tfm = tfms_from_model(arch, sz, aug_tfms=transforms_side_on, max_zoom=1.1)
    data = ImageClassifierData.from_csv(PATH, 'train', f'{PATH}labels.csv', val_idxs = val_idx, tfms=tfm,
                                        suffix= '.jpg', test_name= 'test', 
                                        bs=bs, num_workers = 4)
    return data if sz >300 else data.resize(340, 'tmp')

In [6]:
data = getdata(sz, bs) #get the data 

HBox(children=(IntProgress(value=0, max=6), HTML(value='')))

In [ ]:
fn = PATH+data.trn_ds.fnames[0]; fn

In [ ]:
img = PIL.Image.open(fn); img

In [ ]:
img.size

In [ ]:
#need to check image size 
size_d = {k: PIL.Image.open(PATH+k).size for k in data.trn_ds.fnames}

In [ ]:
row_sz, col_sz = list(zip(*size_d.values())) # link the lists of rows and that 

In [ ]:
row_sz = np.array(row_sz); col_sz = np.array(col_sz)

In [ ]:
row_sz[:5]

In [ ]:
plt.hist(row_sz[row_sz<1000])

In [ ]:
learn = ConvLearner.pretrained(arch, data, precompute=True)

In [ ]:
lrf = learn.lr_find()
learn.predict()

In [ ]:
learn.sched.plot_lr()

In [ ]:
learn.sched.plot()

In [ ]:
learn.fit(1e-2, 5)

In [ ]:
from sklearn import metrics

In [7]:
data = getdata(299,bs)
learn = ConvLearner.pretrained(arch, data, precompute=True, ps=0.5)

HBox(children=(IntProgress(value=0, max=6), HTML(value='')))

In [10]:
learn.fit(1e-2, 5)

RuntimeError: While copying the parameter named 0.weight, whose dimensions in the model are torch.Size([4096]) and whose dimensions in the checkpoint are torch.Size([64, 3, 7, 7]).

In [11]:
learn.precompute = False
learn.load('244_init')

In [ ]:
learn.fit(1e-2, 5, cycle_len =1)

In [ ]:
learn.save('244_init')

In [ ]:
learn.load('244_init')

In [12]:
learn.set_data(getdata(299,bs))
learn.freeze()

HBox(children=(IntProgress(value=0, max=6), HTML(value='')))

In [ ]:
learn.fit(1e-2,3, cycle_len=1)

In [ ]:
learn.fit(1e-2, 3, cycle_len = 1, cycle_mult =2)

In [ ]:
log_preds, y = learn.TTA()
probs = np.exp(log_preds)
accuracy(log_preds,y), metrics.log_loss(y, probs)

In [ ]:
learn.save('299pre')

In [ ]:
learn.load('299pre')

In [ ]:
learn.fit(1e-2,1,cycle_len=2)

In [ ]:
learn.save('299_final')


In [ ]:
log_preds, y = learn.TTA()
probs = np.exp(log_preds)
accuracy(log_preds,y), metrics.log_loss(y, probs)

In [ ]:
log_preds, y = learn.TTA(is_test = True)
probs = np.exp(log_preds)
accuracy(log_preds,y), metrics.log_loss(y, probs)

In [ ]:
learn.fit(1e-2, 2, cycle_len = 1,wds=0.025)

In [13]:
learn.freeze()
learn.precompute = False
learn.load('299_final')

In [ ]:
prob = learn.predict(is_test=True)

In [ ]:
test_log_preds,y = learn.TTA(is_test=True)
test_probs = np.exp(test_log_preds)

In [ ]:
probs


In [ ]:
test_probs = np.exp(probs)

In [ ]:
test_probs.shape

In [ ]:
!ls data/dogbreeds/test | wc -l

In [ ]:
log_preds, y = learn.TTA(is_test = True)
probs = np.mean(np.exp(log_preds), 0)

In [ ]:
"""
submit = pd.DataFrame(probs)
submit.columns = data.classes
submit.insert(0,'id',[Id[5:-4] for Id in data.test_ds.fnames]) #skip prefix and suffix
"""
probs2 = np.exp(log_preds)
probs.shape


In [ ]:
submit = pd.DataFrame(probs2)
submit.columns = data.classes
submit.insert(0, 'id', [Id[5:-4] for Id in data.test_dl.dataset.fnames])
submit.shape


In [ ]:
submit.to_csv(f'{PATH}submit.csv',index=False)

In [15]:
learn.set_data(getdata(349,bs//3))

In [ ]:
learn.fit(1e-2, 2)

HBox(children=(IntProgress(value=0, description='Epoch', max=2), HTML(value='')))

  0%|          | 0/431 [00:00<?, ?it/s]

/home/ubuntu/src/anaconda3/envs/fastai/lib/python3.6/site-packages/torch/nn/modules/container.py:67: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


[0.      0.34647 0.25187 0.91618]                            
 72%|███████▏  | 311/431 [07:00<02:42,  1.35s/it, loss=0.347]

In [ ]:
log_preds, y = learn.TTA(is_test = True)
probs = np.mean(np.exp(log_preds), 0)
probs2 = np.exp(log_preds)
probs.shape

In [ ]:
submit = pd.DataFrame(probs2)
submit.columns = data.classes
submit.insert(0, 'id', [Id[5:-4] for Id in data.test_dl.dataset.fnames])
submit.shape
submit.to_csv(f'{PATH}submit.csv',index=False)